In [1]:
import pandas as pd
import numpy as np

In [2]:
df_schools  = pd.read_csv("raw_data/schools_complete.csv")
df_students = pd.read_csv("raw_data/students_complete.csv")

## District Summary

In [25]:
passing_rate = 0.5*(len(df_students[df_students.math_score>65])/len(df_students) + len(df_students[df_students.reading_score>65])/len(df_students))
df_summary = pd.DataFrame({"Summary Item":
                           ['Total schools', 'Total students','Total budget','Average Math Score',
                           'Average Reading Score','Percentage of student passing Math (>65)',
                            'Percentage of student passing Reading (>65)','Overall passing rate'],
                          "Data":['{:02d}'.format(len(df_schools)),len(df_students),'$'+'{:,}'.format(df_schools['budget'].sum()),
                                  df_students['math_score'].mean(),df_students['reading_score'].mean(),
                                  '{:4.2f}'.format(100*len(df_students[df_students.math_score>65])/len(df_students))+"%",
                                  '{:4.2f}'.format(100*len(df_students[df_students.reading_score>65])/len(df_students))+"%",
                                  '{:4.2f}'.format(100*passing_rate)+"%"]})

In [26]:
df_summary[['Summary Item','Data']]

,Summary Item,Data
0,Total schools,15
1,Total students,39170
2,Total budget,"$24,649,428"
3,Average Math Score,78.9854
4,Average Reading Score,81.8778
5,Percentage of student passing Math (>65),83.11%
6,Percentage of student passing Reading (>65),94.26%
7,Overall passing rate,88.69%


## School Summary

In [27]:
df_schools = df_schools.sort_values('name').reset_index(drop=True)

In [7]:
df_student_count = pd.DataFrame(df_students.groupby('school').count()['name'])
df_student_count = df_student_count.reset_index(drop=False)
df_student_count.columns = ['name','Number_of_students']

In [8]:
df_schools_2= df_schools.set_index('name').join(df_student_count.set_index('name'))
df_schools_2 = df_schools_2.reset_index(drop=False)
df_schools_2['Budget per student']=df_schools_2.budget/df_schools_2.Number_of_students
df_mean_math = pd.DataFrame(df_students.groupby('school')['math_score'].mean()).reset_index(drop=False)
df_mean_math.columns = ['name','mean_math_score']
df_schools_3 = df_schools_2.set_index('name').join(df_mean_math.set_index('name'))
df_schools_3 = df_schools_3.reset_index(drop=False)

In [52]:
df_mean_reading = pd.DataFrame(df_students.groupby('school')['reading_score'].mean()).reset_index(drop=False)
df_mean_reading.columns = ['name','mean_reading_score']
df_schools_4 = df_schools_3.set_index('name').join(df_mean_reading.set_index('name'))
df_schools_4 = df_schools_4.reset_index(drop=False)

In [70]:
# Need the percentage of students that are passing math in each school
# 1. Get the number of students in each school
#    df_schools_4['Number_of_students']
# 2. In each of those school, get the number of students that have passed math 
number_students_passing = df_students[df_students.math_score>65].groupby('school').count()
df_schools_4['Percentage_passing_math']=number_students_passing.reset_index()['math_score']/df_schools_4['Number_of_students']

In [72]:
number_students_passing_reading = df_students[df_students.reading_score>65].groupby('school').count()
df_schools_4['Percentage_passing_reading']=number_students_passing_reading.reset_index()['reading_score']/df_schools_4['Number_of_students']

In [75]:
df_schools_4['Overall Passing Rate'] = 0.5*(df_schools_4['Percentage_passing_reading']+df_schools_4['Percentage_passing_math'])

In [76]:
df_schools_4

,name,School ID,type,size,budget,Number_of_students,Budget per student,mean_math_score,mean_reading_score,Percentage_passing_math,Percentage_passing_reading,Overall Passing Rate
0,Bailey High School,7,District,4976,3124928,4976,628.0,77.048432,81.033963,0.756029,0.919011,0.837520
1,Cabrera High School,6,Charter,1858,1081356,1858,582.0,83.061895,83.975780,1.000000,1.000000,1.000000
2,Figueroa High School,1,District,2949,1884411,2949,639.0,76.711767,81.158020,0.748728,0.918956,0.833842
3,Ford High School,13,District,2739,1763916,2739,644.0,77.102592,80.746258,0.761957,0.907996,0.834976
4,Griffin High School,4,Charter,1468,917500,1468,625.0,83.351499,83.816757,1.000000,1.000000,1.000000
5,Hernandez High School,3,District,4635,3022020,4635,652.0,77.289752,80.934412,0.754693,0.914563,0.834628
6,Holden High School,8,Charter,427,248087,427,581.0,83.803279,83.814988,1.000000,1.000000,1.000000
7,Huang High School,0,District,2917,1910635,2917,655.0,76.629414,81.182722,0.752828,0.917724,0.835276
8,Johnson High School,12,District,4761,3094650,4761,650.0,77.072464,80.966394,0.753833,0.920605,0.837219
9,Pena High School,9,Charter,962,585858,962,609.0,83.839917,84.044699,1.000000,1.000000,1.000000


In [81]:
df_schools_top = df_schools_4.sort_values('Overall Passing Rate',ascending=False)

In [84]:
df_schools_top.head(5)

,name,School ID,type,size,budget,Number_of_students,Budget per student,mean_math_score,mean_reading_score,Percentage_passing_math,Percentage_passing_reading,Overall Passing Rate
1,Cabrera High School,6,Charter,1858,1081356,1858,582.0,83.061895,83.975780,1.0,1.0,1.0
4,Griffin High School,4,Charter,1468,917500,1468,625.0,83.351499,83.816757,1.0,1.0,1.0
6,Holden High School,8,Charter,427,248087,427,581.0,83.803279,83.814988,1.0,1.0,1.0
9,Pena High School,9,Charter,962,585858,962,609.0,83.839917,84.044699,1.0,1.0,1.0
11,Shelton High School,2,Charter,1761,1056600,1761,600.0,83.359455,83.725724,1.0,1.0,1.0


In [85]:
df_schools_bottom = df_schools_4.sort_values('Overall Passing Rate')

In [86]:
df_schools_bottom.head()

,name,School ID,type,size,budget,Number_of_students,Budget per student,mean_math_score,mean_reading_score,Percentage_passing_math,Percentage_passing_reading,Overall Passing Rate
2,Figueroa High School,1,District,2949,1884411,2949,639.0,76.711767,81.158020,0.748728,0.918956,0.833842
5,Hernandez High School,3,District,4635,3022020,4635,652.0,77.289752,80.934412,0.754693,0.914563,0.834628
3,Ford High School,13,District,2739,1763916,2739,644.0,77.102592,80.746258,0.761957,0.907996,0.834976
7,Huang High School,0,District,2917,1910635,2917,655.0,76.629414,81.182722,0.752828,0.917724,0.835276
10,Rodriguez High School,11,District,3999,2547363,3999,637.0,76.842711,80.744686,0.755439,0.915229,0.835334


In [106]:
df_students.groupby('grade').mean()['math_score']

grade
10th    78.941483
11th    79.083548
12th    78.993164
9th     78.935659
Name: math_score, dtype: float64

In [108]:
df_students.groupby('grade').mean()['reading_score']

grade
10th    81.874410
11th    81.885714
12th    81.819851
9th     81.914358
Name: reading_score, dtype: float64

In [112]:
print("Max budget per student: " + str(df_schools_4['Budget per student'].max()))
print("Min budget per student: " + str(df_schools_4['Budget per student'].min()))

Max budget per student: 655.0
Min budget per student: 578.0


In [131]:
bins=[550.,575.,600.,625.,660.]
group_bins = ["550 to 575","575 to 600","600 to 625","625 to 660"]

In [135]:
df_schools_4['View Budget per student groups']=pd.cut(df_schools_4['Budget per student'].map(np.float),bins,labels=group_bins)

In [144]:
df_schools_4.groupby('View Budget per student groups').mean()[['mean_math_score','mean_reading_score','Percentage_passing_math','Percentage_passing_reading','Overall Passing Rate']]

,mean_math_score,mean_reading_score,Percentage_passing_math,Percentage_passing_reading,Overall Passing Rate
View Budget per student groups,,,,,
550 to 575,NaN,NaN,NaN,NaN,NaN
575 to 600,83.436210,83.892196,1.000000,1.00000,1.000000
600 to 625,83.595708,83.930728,1.000000,1.00000,1.000000
625 to 660,77.764435,81.326923,0.785438,0.92676,0.856099


In [146]:
print("Max number of student: " + str(df_schools_4['size'].max()))
print("Min number of student: " + str(df_schools_4['size'].min()))

Max number of student: 4976
Min number of student: 427


In [151]:
bins=[400,2000,3500,5000]
group_bins = ["Small (400 to 2000)","Medium (2000 to 3500)","Large (3500 to 5000)"]

In [152]:
df_schools_4['View number of student groups']=pd.cut(df_schools_4['size'],bins,labels=group_bins)

In [154]:
df_schools_4.groupby('View number of student groups').mean()[['mean_math_score','mean_reading_score','Percentage_passing_math','Percentage_passing_reading','Overall Passing Rate']]

,mean_math_score,mean_reading_score,Percentage_passing_math,Percentage_passing_reading,Overall Passing Rate
View number of student groups,,,,,
Small (400 to 2000),83.502373,83.883125,1.000000,1.000000,1.000000
Medium (2000 to 3500),78.429493,81.769122,0.815878,0.936169,0.876024
Large (3500 to 5000),77.063340,80.919864,0.754998,0.917352,0.836175


In [156]:
df_schools_4.groupby('type').mean()[['mean_math_score','mean_reading_score','Percentage_passing_math','Percentage_passing_reading','Overall Passing Rate']]

,mean_math_score,mean_reading_score,Percentage_passing_math,Percentage_passing_reading,Overall Passing Rate
type,,,,,
Charter,83.473852,83.896421,1.000000,1.000000,1.000000
District,76.956733,80.966636,0.754787,0.916298,0.835542
